# TechXchange Japan 2024: さわってみようベクトル・データベース watsonx.dataでRAG体験


生成AIの回答精度を向上させるために、自社内のデータを活用してみましょう！<br>
ベクトル・データベース + 大規模言語モデル（LLM）で構成されるRAGのアーキテクチャーを使えば、自社内の情報で生成AIチャットボットが作成できます。<br>
当ハンズオンでは「TechXchange Japan 2024」に関するデータを使ってRAGを構成し、「TechXchange Japan 2024」に関することを教えてくれるチャットボットを作成します。<br>
(以下ベクトル・データベースはベクトルDBと表記します。)<br>


具体的には大規模言語モデル（LLM）を使用したアプリケーション開発のためのオープンソース・オーケストレーション・フレームワーク[LangChain](https://python.langchain.com/docs/introduction/)を使って、wastosonx.dataのベクトルDB **Milvus**に「TechXchange Japan 2024」に関するデータをロードし、watson.aiで提供されているLLMを使用してRAGを構成し、「TechXchange Japan 2024」のことを回答してくれるChatbotを作ってみます。

ハンズオンは以下の順序で実行します:

1. Excelをベクトル化してベクトルDB Milvusに入れよう！
2. ベクトルDB Milvusに入ったデータで類似検索してみよう!
3. ベクトルDB Milvusとwatsonx.ai LLMでRAGを構成して、質問をしてみよう!
4. **ベクトルDB Milvusとwatsonx.ai LLMでRAGを構成して、チャットアプリを作成してみよう!**[**当notebook**]


このハンズオンのガイドは[**こちら**](https://ibm.biz/20241127-rag-handson)にあります。

## 4. ベクトルDB Milvusとwatsonx.ai LLMでRAGを構成して、チャットアプリを作成してみよう!

**実行は　セルを選択(クリック)して**

- **Windows： Ctrl+Enter**
- **Mac: ⌘ (command) + Enter 　または Ctrl + Enter**

セルの左側に[*]が表示されている場合は実行中です。<br>
[1]のように数字が入っている場合は、実行が完了しています。

### 1. 必要なライブラリーのインストール

In [ ]:
!pip install -U 'ibm-watsonx-ai>=1.1.15'
!pip install -Uq 'langchain>=0.3.3'
!pip install -Uq 'langchain-ibm>=0.3.1'
!pip install -Uq 'langchain-huggingface>=0.1.0'
!pip install -Uq 'langchain-milvus>=0.1.6'
!pip install -Uq 'langchain-community>=0.3.2'
!pip install -Uq 'pymilvus>=2.4.8'
!pip install -U gradio

**インストール終了後、一旦カーネルを再起動してください** <br>

**手順:**
- 上部のメニュー「Karnel」から「Restart Karnel and Clear Outputs of All Cells...」をクリック
- 「Restart Kernel?」 のダイアログが表示されるので、「Restart」をクリック

### 2. apikeyの設定 

- 実行するとテキスト入力ボックスが表示されるので、事前に取得したapikeyをに入れてEnter Keyを押してください

- ハンズオン環境で実行の場合は、`IBM Cloud Service API key`を入力してください。　よくわからない場合は[こちら](https://github.com/IBM/japan-technology/tree/main/techxchange/2024-watsonx-handson-1/01_techzone_use_environments.md)の2-3を参照してください。



In [ ]:
import getpass
apikey = getpass.getpass("apikeyを入れてEnter Keyを押してください:")

### 3. Milvus接続情報の設定

実行すると順に以下の2つのテキスト入力ボックスが表示されるので、事前に取得したMilvus接続情報の値を入力してください。
- milvus GRPC ホストを入れてEnter Keyを押してください →  GRPC ホストの値を入力  

- milvus GRPC ポートを入れてEnter Keyを押してください　→ GRPC ポートの値をを入力


Milvus接続情報の詳細取得手順は[こちら](https://github.com/IBM/japan-technology/tree/main/techxchange/2024-watsonx-handson-1/watsonx_data_get_milvus_info.md) を参照

簡略な接続手順は以下です:
- watsonx.dataの画面を開く
- ナビゲーションメニューから「インフラストラクチャー・マネージャー」を選択
- サービス「Milvus」をクリック
- タイプの下の「接続の詳細を見る」をクリック
- GRPC ホストの値とGRPC ポートの値を取得
  
collection名は`techxchange_line_data`としています。

In [ ]:
milvus_host=input("milvus GRPC ホストを入れてEnter Keyを押してください: ")
milvus_port=input("milvus GRPC ポートを入れてEnter Keyを押してください: ")

In [ ]:
# Milvus接続情報パラメータののセット
my_connection_args ={
 'uri': f'https://{milvus_host}:{milvus_port}', 
 'token': f'ibmlhapikey:{apikey}'
}
my_collection =  'techxchange_line_data'

### 3. watsonx.ai Project idの設定

Watson Studioで実行する場合は、このノートブックが実行されるプロジェクトからProject idを取得します。
Watson Studio以外で実行する場合は、Project idを入力してください。

**Hint**: `project_id` はプロジェクトを表示し、管理タブから `project_id` を取得可能です.

In [ ]:
import os
try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Project idを入力してください (入力後enter): ")

### 5.watsonx.aiのAuthentication用のエンドポイントのURLの設定

Waston Machine Learningのインスタンスを作成したリージョンで決まります。
https://ibm.github.io/watson-machine-learning-sdk/setup_cloud.html#authentication　より

- Dallas: https://us-south.ml.cloud.ibm.com
- London: https://eu-gb.ml.cloud.ibm.com
- Frankfurt: https://eu-de.ml.cloud.ibm.com
- Tokyo: https://jp-tok.ml.cloud.ibm.com

今回はダラスのWaston Machine Learningのインスタンスを使っているので`https://us-south.ml.cloud.ibm.com`を使います。

In [ ]:
watsonx_url = "https://us-south.ml.cloud.ibm.com" #watsonx.aiのAuthentication用のエンドポイントのURL

### 4. 必要ライブラリーのImport

In [ ]:
import pandas as pd
from langchain.schema.document import Document
import json
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_milvus import Milvus
import os
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from langchain_ibm import WatsonxLLM
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough


os.environ["TOKENIZERS_PARALLELISM"] = "false"

### 4. Embeddingモデルの取得
ベクトル化した時と同じモデル`intfloat/multilingual-e5-large`を使います<br>
https://huggingface.co/intfloat/multilingual-e5-large

`TqdmExperimentalWarning`のWarningが表示された場合は無視でよいです。

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

### 5.  ベクトルDB Milvusに接続

In [ ]:
from langchain_milvus import Milvus

vector_db = Milvus(
    embeddings,
    connection_args =my_connection_args,
    collection_name = my_collection
)

### 6.  watsonx.ai LLMの取得　および　プロンプトテンプレートの作成

今回3つ切り替えて試せるようにします。以下の3つを取得し、それに適したプロンプトテンプレートを作成します。

- mixtral-8x7b-instruct-v01
- llama-3-2-3b-instruct
- granite-3-8b-instruct

#### mixtral-8x7b-instruct-v01

In [ ]:
custom_llm = {}
rag_prompt = {}
no_rag_prompt = {}

# 使用するLLMのパラメータ: mixtral-8x7b-instruct-v0
key = 'mixtral-8x7b-instruct-v01'
generate_params = {
    GenParams.MAX_NEW_TOKENS: 16384,
    GenParams.MIN_NEW_TOKENS: 0,
    GenParams.DECODING_METHOD: "greedy",
    GenParams.REPETITION_PENALTY: 1
}

# LangChainで使うllm: mixtral-8x7b-instruct-v0
custom_llm[key] = WatsonxLLM(
    model_id="mistralai/mixtral-8x7b-instruct-v01",
    url=watsonx_url,
    apikey=apikey,
    project_id=project_id,
    params=generate_params,
)

# LangChainで使うPrompt(RAGあり): mixtral-8x7b-instruct-v0
template = """<s> [INST] 
あなたは親切で、礼儀正しく、誠実なアシスタントです。常に安全を保ちながら、できるだけユーザーの役に立つように詳しく回答してください。
回答には、有害、非倫理的、⼈種差別的、性差別的、有毒、危険、または違法なコンテンツを含めてはいけません。回答は社会的に偏⾒がなく、本質的に前向きなものであることを確認してください。
質問が意味をなさない場合、または事実に⼀貫性がない場合は、正しくないことに答えるのではなく、その理由を説明してください。質問の答えがわからない場合は、誤った情報を共有しないでください。
questionに答えるために、以下のcontextを使用し必ず日本語でanswerを作成してください。
必ず⽇本語の文章で回答してください。知ったかぶりをしないでください。
回答を書くときは、context内の単語をできるだけ使⽤してください。context中に質問に対する回答が無い低い場合は、「文書中に質問に対する回答が明記されていません。」とだけ回答してください。「文書中に質問に対する回答が明記されていません。」と回答した場合、そこで回答を終わりにしてください。
contextの内容がブランクの場合、「文書中に質問に対する回答が明記されていません。」とだけ回答してください。
セッションについて回答する場合は、タイトル、開始時刻、終了時刻、会場、概要、レベルを回答してください。回答が200文字以上の場合、回答はなるべく箇条書きを含めてわかりやすく回答してください。
[/INST]
</s>
<s> [INST] 質問が質問の文章ではなく意味がわからない場合[/INST]もう少し詳しく説明していただけますか？</s>

context: {context}
question: {question}
answer: 
"""

# Prompt Templateを作成します
rag_prompt[key] = PromptTemplate.from_template(template)


# LangChainで使うPrompt(RAGなし): mixtral-8x7b-instruct-v0
template = """<s> [INST] 
必ず⽇本語の文章で回答してください。知ったかぶりをしないでください。。
あなたは親切で、礼儀正しく、誠実なアシスタントです。常に安全を保ちながら、できるだけユーザーの役に立つように詳しく回答してください。
回答には、有害、非倫理的、⼈種差別的、性差別的、有毒、危険、または違法なコンテンツを含めてはいけません。回答は社会的に偏⾒がなく、本質的に前向きなものであることを確認してください。
質問が意味をなさない場合、または事実に⼀貫性がない場合は、正しくないことに答えるのではなく、その理由を説明してください。質問の答えがわからない場合は、誤った情報を共有しないでください。
[/INST]
</s>
<s> [INST] 質問が質問の文章ではなく意味がわからない場合[/INST]もう少し詳しく説明していただけますか？</s>
question: {question}
answer: 
"""

# Prompt Templateを作成します
no_rag_prompt[key] = PromptTemplate.from_template(template)

#### llama-3-2-3b-instruct

In [ ]:
# 使用するLLMのパラメータ:llama-3-2-3b-instruct
key = 'llama-3-2-3b-instruct'
generate_params = {
    GenParams.MAX_NEW_TOKENS: 8192,
    GenParams.MIN_NEW_TOKENS: 0,
    GenParams.DECODING_METHOD: "greedy",
    GenParams.REPETITION_PENALTY: 1
}

# LangChainで使うllm: llama-3-2-3b-instruct
custom_llm[key] = WatsonxLLM(
    model_id="meta-llama/llama-3-2-3b-instruct",
    url=watsonx_url,
    apikey=apikey,
    project_id=project_id,
    params=generate_params,
)

# LangChainで使うPrompt(RAGあり): llama-3-2-3b-instruct
template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
あなたは親切で、礼儀正しく、誠実なアシスタントです。常に安全を保ちながら、できるだけユーザーの役に立つように詳しく回答してください。
回答には、有害、非倫理的、⼈種差別的、性差別的、有毒、危険、または違法なコンテンツを含めてはいけません。回答は社会的に偏⾒がなく、本質的に前向きなものであることを確認してください。
質問が意味をなさない場合、または事実に⼀貫性がない場合は、正しくないことに答えるのではなく、その理由を説明してください。質問の答えがわからない場合は、誤った情報を共有しないでください。
questionに答えるために、以下のcontextを使用し必ず日本語でanswerを作成してください。
必ず⽇本語の文章で回答してください。知ったかぶりをしないでください。
回答を書くときは、context内の単語をできるだけ使⽤してください。context中に質問に対する回答が無い低い場合は、「文書中に質問に対する回答が明記されていません。」とだけ回答してください。「文書中に質問に対する回答が明記されていません。」と回答した場合、そこで回答を終わりにしてください。
contextの内容がブランクの場合、「文書中に質問に対する回答が明記されていません。」とだけ回答してください。
セッションについて回答する場合は、タイトル、開始時刻、終了時刻、会場、概要、レベルを回答してください。回答が200文字以上の場合、回答はなるべく箇条書きを含めてわかりやすく回答してください。
<|eot_id|><|start_header_id|>user<|end_header_id|>

context: {context} <|eot_id|><|start_header_id|>user<|end_header_id|>

{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

# Prompt Templateを作成します
rag_prompt[key] = PromptTemplate.from_template(template)

# LangChainで使うPrompt(RAGなし): llama-3-2-3b-instruct
template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
あなたは親切で、礼儀正しく、誠実なアシスタントです。常に安全を保ちながら、できるだけユーザーの役に立つように詳しく回答してください。
回答には、有害、非倫理的、⼈種差別的、性差別的、有毒、危険、または違法なコンテンツを含めてはいけません。回答は社会的に偏⾒がなく、本質的に前向きなものであることを確認してください。
必ず⽇本語の文章で回答してください。知ったかぶりをしないでください。
質問が意味をなさない場合、または事実に⼀貫性がない場合は、正しくないことに答えるのではなく、その理由を説明してください。質問の答えがわからない場合は、誤った情報を共有しないでください。<|eot_id|><|start_header_id|>user<|end_header_id|>

{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

# Prompt Templateを作成します
no_rag_prompt[key] = PromptTemplate.from_template(template)

####  granite-3-8b-instruct

In [ ]:
# 使用するLLMのパラメータ: granite-3-8b-instruct
key = 'granite-3-8b-instruct'
generate_params = {
    GenParams.MAX_NEW_TOKENS: 4096,
    GenParams.MIN_NEW_TOKENS: 0,
    GenParams.DECODING_METHOD: "greedy",
    GenParams.REPETITION_PENALTY: 1
}

# LangChainで使うllm: granite-3-8b-instruct
custom_llm[key] = WatsonxLLM(
    model_id="ibm/granite-3-8b-instruct",
    url=watsonx_url,
    apikey=apikey,
    project_id=project_id,
    params=generate_params,
)

# LangChainで使うPromptt(RAGあり): granite-3-8b-instruct
template = """<|start_of_role|>system<|end_of_role|>なたは2024年にIBMが開発したAI言語モデル「Granite」です。あなたは親切で、礼儀正しく、誠実なアシスタントです。常に安全を保ちながら、できるだけユーザーの役に立つように詳しく回答してください。
回答には、有害、非倫理的、⼈種差別的、性差別的、有毒、危険、または違法なコンテンツを含めてはいけません。回答は社会的に偏⾒がなく、本質的に前向きなものであることを確認してください。
質問が意味をなさない場合、または事実に⼀貫性がない場合は、正しくないことに答えるのではなく、その理由を説明してください。質問の答えがわからない場合は、誤った情報を共有しないでください。
questionに答えるために、以下のcontextを使用し必ず日本語でanswerを作成してください
必ず⽇本語の文章で回答してください。知ったかぶりをしないでください。
回答を書くときは、context内の単語をできるだけ使⽤してください。context中に質問に対する回答が無い低い場合は、「文書中に質問に対する回答が明記されていません。」とだけ回答してください。「文書中に質問に対する回答が明記されていません。」と回答した場合、そこで回答を終わりにしてください。
contextの内容がブランクの場合、「文書中に質問に対する回答が明記されていません。」とだけ回答してください。
セッションについて回答する場合は、タイトル、開始時刻、終了時刻、会場、概要、レベルを回答してください。回答が200文字以上の場合、回答はなるべく箇条書きを含めてわかりやすく回答してください。
context: {context}
<|end_of_text|>
<|start_of_role|>user<|end_of_role|>question:{question}<|end_of_text|>
<|start_of_role|>assistant<|end_of_role|>
"""


# Prompt Templateを作成します
rag_prompt[key] = PromptTemplate.from_template(template)

# LangChainで使うPromptt(RAGなし): granite-3-8b-instruct
template = """<|start_of_role|>system<|end_of_role|>あなたは2024年にIBMが開発したAI言語モデル「Granite」です。
あなたは親切で、礼儀正しく、誠実なアシスタントです。常に安全を保ちながら、できるだけユーザーの役に立つように詳しく回答してください。
回答には、有害、非倫理的、⼈種差別的、性差別的、有毒、危険、または違法なコンテンツを含めてはいけません。回答は社会的に偏⾒がなく、本質的に前向きなものであることを確認してください。
必ず⽇本語の文章で回答してください。知ったかぶりをしないでください。
質問が意味をなさない場合、または事実に⼀貫性がない場合は、正しくないことに答えるのではなく、その理由を説明してください。質問の答えがわからない場合は、誤った情報を共有しないでください。
回答が200文字以上の場合、回答はなるべく箇条書きを含めてわかりやすく回答してください。
<|end_of_text|>
<|start_of_role|>user<|end_of_role|>question:{question}<|end_of_text|>
<|start_of_role|>assistant<|end_of_role|>
"""

# Prompt Templateを作成します
no_rag_prompt[key] = PromptTemplate.from_template(template)


### 7. Retrieverの作成
ここでは「 [3. ベクトル・データベース Milvusとwatsonx.ai LLMでRAGを構成して、質問をしてみよう!](https://github.com/IBM/japan-technology/blob/main/techxchange/2024-watsonx-handson-1/notebooks/techxchange_handson_03_RAG.ipynb)」の **「閾値以上の類似スコアを持つ文書のみを返す」CustomRetriever**を使います.

類似スコアの閾値と検索件数をGUIで指定できるようにします。


In [ ]:
# 閾値以上の類似スコアを持つ文書のみを返す」CustomRetrieverを作成
# asyncの方は省略
from langchain.schema.vectorstore import VectorStoreRetriever
from langchain.schema.document import Document
from langchain.callbacks.manager import (
    CallbackManagerForRetrieverRun,
)
from typing import List

class CustomRetriever(VectorStoreRetriever):
    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        top_k = self.search_kwargs.get("k", 4)
        docs_and_similarities = self.vectorstore.similarity_search_with_score(query, k=top_k)
                
        threshold = self.search_kwargs.get("score_threshold", 0)
       
        return [doc for doc, score in docs_and_similarities if score > threshold and score < 1]

In [ ]:
# 類似スコアの閾値と検索件数を指定して、Retrieverを作成する関数
def get_Retriever(vectorstore, search_score_flag, score, k):
    if search_score_flag == "On": # 類似スコアの閾値を指定する場合
        return CustomRetriever(vectorstore = vectorstore, search_kwargs={"score_threshold": score, "k":k})
    else: # 類似スコアの閾値を指定しない場合
        return vectorstore.as_retriever(search_kwargs={"k": k})   

### 8. テストしてみます

- 作成したRetrieverの類似検索テストしてみます
- LLMを指定してChainを作成し、質問してみます

In [ ]:
# 作成したRetrieverの類似検索テスト

query="RAGに関するセッションを教えてください"

retriever = get_Retriever(vector_db, "On", 0.83, 20)
docs = retriever.invoke(query)
# print(docs)
for doc in docs:
    print({"content": doc.page_content, "metadata": doc.metadata} )
    print("---------")

In [ ]:
# Chainの作成
# 1つKeyを指定してください

# key = 'llama-3-2-3b-instruct'
# key = 'mixtral-8x7b-instruct-v01'
key = 'granite-3-8b-instruct'
retriever = get_Retriever(vector_db, "On", 0.83, 10)
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt[key]
    | custom_llm[key]
)

In [ ]:
# 質問してみます

query="RAGに関するセッションの詳細を教えてください"
# query="watsonx.dataに関するセッションの詳細を教えてください"
# query="TechXchangeについて概要を教えてください"
# query="量子コンピューター関連のセッションを教えてください"
print(rag_chain.invoke(query))

### 9. gradioでGUI作成

[gradio](https://www.gradio.app/)を使ってGUIを作り、いろいろな条件でRAG構成を試してみましょう！

尚、会話履歴を読むようにしていないので、毎回フルセンテンスで質問するようにお願いします。

プロンプトを変更して試してもOKです。

 public URLがインターネット上のURLになります。このLinkは72時間保持されます. 


share_flag :
- Watson Studio仕様の場合はTrue
- Trueにするとインターネットに公開されるので、機密情報がある場合はTrueにしないようにしてください。
- PC上のnotebookで実行する場合は、FalseにしてLocalPCでの実行が可能です。

下のセルを実行するとGUIが表示されます。<br>
public URLをクリックして、Webブラウザーで開いて操作してみましょう。<br>
エラーメッセージ等はnotebookに出ますので、エラーが発生した場合はnotebookのセル出力のGUIの下の方を参照してください。<br>


In [ ]:
import gradio as gr
from langchain_core.output_parsers import StrOutputParser

# Watson Studio仕様の場合はTrue
# インターネットに公開されるので、機密情報がある場合はTrueにしないようにしてください。
# PC上のnotebookで実行する場合は、FalseにしてLocalPCでの実行が可能です。

share_flag = True
# share_flag = False

def chat_streaming(message, history, RAG_flag, model_name, search_k, search_score_flag, search_score ):
    gr_retriever = get_Retriever(vector_db, search_score_flag, search_score, search_k)
    gr_rag_chain = None
    if RAG_flag == "Off":
        gr_rag_chain = (
            {"question": RunnablePassthrough()}
            | no_rag_prompt[model_name]
            | custom_llm[model_name] 
        )
        print("RAG OFF")
    else:
        gr_rag_chain = (
            {"context": gr_retriever, "question": RunnablePassthrough()}
            | rag_prompt[model_name]
            | custom_llm[model_name] 
        )
      
    
    if message is not None:
        partial_message = ""
        for response in gr_rag_chain.stream(message):
            partial_message += response    
            yield partial_message 

def change_search_score(flag):
    if flag == "On":
        interactive_flag = True
    else:
        interactive_flag = False
    return gr.update(interactive = interactive_flag, visible= interactive_flag)    

CSS = """#row1 {flex-grow: 1; align-items: unset;}
    .form {height: fit-content;}""" 

with gr.Blocks(fill_height=True,  css=CSS) as demo:
    gr.Markdown("## TechXchange Japan Q&A")

    with gr.Row(equal_height=False, elem_id="row1"):
        with gr.Column(variant="panel", scale=1):
            with gr.Accordion("設定"):
                RAG_flag = gr.Radio(choices=["On", "Off"], type="value", label="RAG構成", value="On", interactive = True)
                model_name = gr.Dropdown(["mixtral-8x7b-instruct-v01", "granite-3-8b-instruct", "llama-3-2-3b-instruct"], label="LLM Model", value="mixtral-8x7b-instruct-v01")
                search_k = gr.Number(label="類似検索の結果取得数(k) 最大15",minimum=1, maximum=15, value=10, interactive = True )
                search_score_flag  = gr.Radio(["On", "Off"], label="類似度スコアの閾値を設定", value="On", interactive = True)
                search_score = gr.Slider(minimum=0, maximum=1, step=0.01, value=0.83, label="類似度スコアの閾値" ,interactive = True, visible=True)
            gr.Markdown("Graniteは最大トークン数が少ないため,　取得数を上げたり類似度スコアの閾値を下げるとエラーになることがあります。その場合は取得数を少なくしたり類似度スコアの閾値上げて調整してください。")
            
        with gr.Column(scale=2):
            chatbot = gr.ChatInterface(fn=chat_streaming, type="messages", 
                                       additional_inputs =[RAG_flag, model_name, search_k, search_score_flag, search_score], 
                                       title="TechXchange Bot", fill_height=True)
          
                                    
    search_score_flag.change(fn=change_search_score, inputs=search_score_flag, outputs=search_score)
  

demo.launch(share=share_flag)

### これで「ベクトルDB Milvusとwatsonx.ai LLMでRAGを構成して、チャットアプリを作成してみよう!」は完了です。<br>


#### Notebookを保存する場合は、右上の保存アイコンをクリックして保存してください。

- <img width="400" alt="" src="https://github.com/IBM/japan-technology/blob/main/techxchange/2024-watsonx-handson-1/images/save_notebook.jpg?raw=true">
<br>
<br>

#### プロジェクトの画面に戻る場合は、右上のプロジェクト名をクリックしてください。

- <img width="400" alt="" src="https://github.com/IBM/japan-technology/blob/main/techxchange/2024-watsonx-handson-1/images/return_to_project.jpg?raw=true">
<br>
<br>

#### Notebookを開いたままでプロジェクトの画面を表示するには、上部のプロジェクト名を右クリックし、「新しいタブで開く」でプロジェクトを新しいタブで開いてください。

- <img width="500" alt="" src="https://github.com/IBM/japan-technology/blob/main/techxchange/2024-watsonx-handson-1/images/open_project_from_notebook.jpg?raw=true">
<br>
<br>

### ハンズオンはすべて完了しました！　お疲れ様でした！